In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import cifar10
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

import warnings
warnings.filterwarnings('ignore')

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
def preprocess_data(X, y):
    X = X.astype('float32') / 255.0
    y = to_categorical(y, num_classes=10)
    return X, y

In [ ]:
x_train, y_train = preprocess_data(X_train, y_train)
x_test, y_test = preprocess_data(X_test, y_test)

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                  patience=10, 
                                                  restore_best_weights=True)

history = model.fit(x_train, y_train, epochs=5, 
                    batch_size=32, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
y_test_bin = label_binarize(y_test, classes=list(range(10)))
y_pred_proba = model.predict(x_test)

In [ ]:
fpr, tpr, _ = roc_curve(y_test_bin[:, 0], y_pred_proba[:, 0])
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Class 0 ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Class 0')
plt.legend(loc="lower right")
plt.show()